In [294]:
import cv2
from ipywidgets import Video
import ipywidgets as widgets
from IPython.display import display
import numpy as np

In [295]:
video_capture = cv2.VideoCapture('Chess_Data\\chess_easy1.mp4')

### Full detection

In [296]:
import cv2
import numpy as np

def find_closest_points_to_corners(points, corners):
    closest_points = []
    for corner in corners:
        distances = np.linalg.norm(points - corner, axis=1)
        closest_point_index = np.argmin(distances)
        closest_points.append(points[closest_point_index])
    return np.array(closest_points)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    frame_with_boxes = frame.copy()
    dst = cv2.cornerHarris(gray, 3, 3, 0.04)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())

    # Find corners closest to image corners
    image_corners = np.array([[0, 0], [0, frame.shape[1]], [frame.shape[0], 0], [frame.shape[0], frame.shape[1]]])
    closest_corners = find_closest_points_to_corners(corners, image_corners)

    # Draw the resulting trapezoid
    #cv2.polylines(frame_with_boxes, [closest_corners], isClosed=True, color=(0, 255, 0), thickness=2)
    #frame_with_boxes[dst > 0.01 * dst.max()] = [0, 0, 255]
    for corner in closest_corners:
        cv2.circle(frame_with_boxes, tuple(corner[::-1]), 5, (0, 0, 255), -1)
    print(closest_corners)

    # Display the result
    cv2.imshow('Frame with Trapezoid', frame_with_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

[[  44  423]
 [  50 1420]
 [1034  276]
 [1040 1506]]
[[  46  407]
 [ 138 1389]
 [1033  276]
 [1039 1506]]
[[  44  422]
 [ 139 1389]
 [1034  276]
 [1039 1506]]
[[  44  422]
 [  50 1419]
 [1033  275]
 [1039 1506]]
[[  44  422]
 [  50 1419]
 [1033  275]
 [1039 1506]]
[[  43  423]
 [ 138 1389]
 [1033  276]
 [1038 1506]]
[[  43  423]
 [  50 1419]
 [1033  275]
 [1040 1506]]
[[  43  422]
 [ 138 1389]
 [1033  276]
 [1040 1506]]
[[  44  422]
 [ 138 1389]
 [1033  276]
 [1040 1506]]
[[  47  407]
 [ 139 1389]
 [1033  275]
 [1042 1505]]
[[  46  407]
 [ 140 1390]
 [1034  275]
 [1041 1506]]
[[  47  407]
 [ 140 1390]
 [1034  276]
 [1041 1506]]
[[  48  407]
 [ 141 1390]
 [1035  275]
 [1042 1507]]
[[  48  408]
 [ 142 1390]
 [1036  276]
 [1042 1507]]
[[  49  408]
 [  51 1416]
 [1037  276]
 [1044 1507]]
[[  50  408]
 [ 144 1390]
 [1038  276]
 [1044 1507]]
[[  51  408]
 [ 144 1390]
 [1039  276]
 [1046 1507]]
[[  52  408]
 [ 146 1390]
 [1040  276]
 [1047 1507]]
[[  52  408]
 [ 146 1390]
 [1041  277]
 [1047 

In [297]:
while True:   
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    frame_with_boxes = frame.copy()
    total_area = frame.shape[0] * frame.shape[1]
    giant_contour = np.vstack(contours)

    # Calculate the bounding box of the giant contour
    x, y, w, h = cv2.boundingRect(giant_contour)
    dst = cv2.cornerHarris(gray, 3, 3, 0.04)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())

    lc = np.min(corners[:, 0])
    tc = np.min(corners[:, 1])
    rc = np.max(corners[:, 0])
    bc = np.max(corners[:, 1])
    frame_with_boxes[dst > 0.01 * dst.max()] = [0, 0, 255]

    # Draw the resulting bounding box
    cv2.rectangle(frame_with_boxes, (tc, lc), (bc, rc), (0, 255, 0), 2)

    # Display the result
    cv2.imshow('Frame with Giant Bounding Box', frame_with_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

In [298]:
while True:   
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    frame_with_boxes = frame.copy()
    total_area = frame.shape[0] * frame.shape[1]
    giant_contour = np.vstack(contours)

    # Calculate the bounding box of the giant contour
    x, y, w, h = cv2.boundingRect(giant_contour)
    dst = cv2.cornerHarris(gray, 3, 3, 0.04)
    dst = cv2.dilate(dst, None)
    corners = np.argwhere(dst > 0.01 * dst.max())

    lc = np.min(corners[:, 0])
    tc = np.min(corners[:, 1])
    rc = np.max(corners[:, 0])
    bc = np.max(corners[:, 1])
    frame_with_boxes[dst > 0.01 * dst.max()] = [0, 0, 255]

    # Draw the resulting bounding box
    cv2.rectangle(frame_with_boxes, (tc, lc), (bc, rc), (0, 255, 0), 2)

    # Display the result
    cv2.imshow('Frame with Giant Bounding Box', frame_with_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

In [299]:
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    frame_with_boxes = frame.copy()
    total_area = frame.shape[0] * frame.shape[1]
    threshold_lower = 0
    threshold_upper = np.inf
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > threshold_lower and area<threshold_upper:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame_with_boxes, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Display the result
    dst = cv2.cornerHarris(gray, 3, 3, 0.04)
    # result is dilated for marking the corners, not important
    dst = cv2.dilate(dst, None)

    # Threshold for an optimal value, it may vary depending on the image.
    frame[dst > 0.01 * dst.max()] = [0, 0, 255]
    cv2.imshow('Frame with Boxes', frame_with_boxes)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
video_capture.release()
cv2.destroyAllWindows()

In [300]:
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Use HoughLinesP to detect lines in the edges
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=10)

    # Draw the detected lines on the original frame
    frame_with_lines = frame.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Display the frame with detected lines
    cv2.imshow('Frame with Lines', frame_with_lines)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close windows
video_capture.release()
cv2.destroyAllWindows()

### Keypoint Detection

In [301]:
sift = cv2.SIFT_create()

while True:   
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    sift_kp1, sift_desc1 = sift.detectAndCompute(gray, None)
    img_kp1 = cv2.drawKeypoints(
        frame, sift_kp1, None, flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT
    )

    cv2.imshow('Video Frame', img_kp1)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

### Corner Detection

In [302]:
while True:   
    ret, frame = video_capture.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    dst = cv2.cornerHarris(gray, 3, 3, 0.04)
    # result is dilated for marking the corners, not important
    dst = cv2.dilate(dst, None)

    # Threshold for an optimal value, it may vary depending on the image.
    frame[dst > 0.01 * dst.max()] = [0, 0, 255]

    cv2.imshow('Video Frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()